In [1]:
'''import all the required modules'''

import os
import pandas as pd
import regex as re
from tqdm.notebook import tqdm
import PIL
import PIL.Image
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from tqdm.notebook import tqdm
%matplotlib inline

'Newfolder created with particpent name'
session_number = "001"
participant_name ="user2"
root_path = './'
os.chdir(root_path)
folders = ['{}_heatmap'.format(participant_name), '{}_fixations'.format(participant_name), '{}_scanpath'.format(participant_name)]

for folder in folders:
    os.mkdir(os.path.join(root_path, folder))

#'Newfolder created with particpent name'
#path='./'
#os.chdir(path)
#Newfolder=participant_name
#os.makedirs(Newfolder,exist_ok=True)

'''resolution of the screen , device where eye tracking test takes place'''

screen_width = 1920
screen_height = 1080

''' df_eyetracking - - - Contains participants's eye tracking data.
    l_valid        - - - left_gaze_point_validity.
    r_valid        - - - right_gaze_point_validity.
    l_display      - - - left_gaze_point_on_display_area. 
    r_display      - - - right_gaze_point_on_display_area.
    time           - - - system_time_stamp.
    l_diameter     - - - left_pupil_diameter.
    r_diameter     - - - right_pupil_diameter.
    
    df_answers     - - - Contains answers given by participant.
    df_cutting     - - - Contains time spent on each images. '''

df_eyetracking = pd.read_csv("./data/eye_tracking_data_" + session_number + " _ " + participant_name + ".csv", sep=";")
df_eyetracking.columns = ["l_valid", "r_valid", "l_display", "r_display", "time", "l_diameter", "r_diameter"]
df_answers = pd.read_csv("./data/study_answers_" + session_number + " _ " + participant_name + ".csv")
df_answers = df_answers.iloc[:, 1:]
df_cutting = pd.read_csv("./data/study_cutting_data_" + session_number + " _ " + participant_name + ".csv")
df_cutting = df_cutting.iloc[:, 1:]


df_eyetracking["l_display_x"] = df_eyetracking["l_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[0])*screen_width) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
df_eyetracking["l_display_y"] = df_eyetracking["l_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[1])*screen_height) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
df_eyetracking["r_display_x"] = df_eyetracking["r_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[0])*screen_width) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
df_eyetracking["r_display_y"] = df_eyetracking["r_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[1])*screen_height) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)

df_eyetracking = df_eyetracking.drop("l_display", axis=1)
df_eyetracking = df_eyetracking.drop("r_display", axis=1)
df_eyetracking.head()

,l_valid,r_valid,time,l_diameter,r_diameter,l_display_x,l_display_y,r_display_x,r_display_y
0,0,0,1810414829,NaN,NaN,0.0,0.0,0.0,0.0
1,0,0,1810423159,NaN,NaN,0.0,0.0,0.0,0.0
2,1,1,1810431517,3.783936,3.968201,677.0,530.0,674.0,522.0
3,1,1,1810439841,NaN,NaN,675.0,519.0,670.0,520.0
4,1,1,1810448159,NaN,NaN,679.0,525.0,668.0,533.0


In [2]:
#time conversion
eye_start = df_eyetracking.loc[0, "time"]
print (eye_start)
df_eyetracking["time"] = df_eyetracking["time"] - eye_start
df_eyetracking["time"] = df_eyetracking["time"] / 1000000
time=df_eyetracking['time']
time

1810414829


0           0.000000
1           0.008330
2           0.016688
3           0.025012
4           0.033330
             ...    
118208    984.961164
118209    984.969498
118210    984.977804
118211    984.986135
118212    984.994501
Name: time, Length: 118213, dtype: float64

In [3]:
def fixation_detection(x_values, y_values, time, maxdist=25, mindur=0.020):

    """Detects fixations, defined as consecutive samples with an inter-sample
    distance of less than a set amount of pixels (disregarding missing data)

    arguments

    x-numpy array of x positions
    y-numpy array of y positions
    time-numpy array of EyeTribe timestamps

    keyword arguments

    missing - value to be used for missing data (default = 0.0)
    maxdist - maximal inter sample distance in pixels (default = 25)
    mindur - minimal duration of a fixation in milliseconds; detected
    fixation cadidates will be disregarded if they are below
    this duration (default = 100)

    returns
    Sfix, Efix

    Sfix - list of lists, each containing [starttime]
    Efix - list of lists, each containing [starttime, endtime, duration, endx, endy]
    """

    # empty list to contain data
    Sfix = []
    Efix = []
    
    # loop through all coordinates
    si = 0
    fixstart = False
    for i in range(1,len(x_values)):
        
        # calculate Euclidean distance from the current fixation coordinate
        # to the next coordinate
        
        squared_distance = ((x_values[si]-x_values[i])**2 + (y_values[si]-y_values[i])**2)
        dist = 0.0
        if squared_distance > 0:
            dist = squared_distance**0.5
            
        # check if the next coordinate is below maximal distance
        
        if dist <= maxdist:
            # start a new fixation
            si = 0 + i
            fixstart = False
            Sfix.append([time[i]])
        elif dist > maxdist and not fixstart:
            
            # end the current fixation
            
            fixstart = False
            if Sfix:
                

                # only store the fixation if the duration is ok

                if time[i-1]-Sfix[-1][0] >= mindur:
                    Efix.append([Sfix[-1][0], time[i-1], time[i-1]-Sfix[-1][0], x_values[si], y_values[si]])

                # delete the last fixation start if it was too short

                else:
                    Sfix.pop(-1)
                si = 0 + i
            else:
                pass
        elif not fixstart:
            si += 1
            
    #add last fixation end (we can lose it if dist > maxdist is false for the last point)
    
        elif len(Sfix) > len(Efix):
            Efix.append([Sfix[-1][0], time[len(x_values)-1], time[len(x_values)-1]-Sfix[-1][0], x_values[si], y_values[si]])
        
    return Sfix, Efix



In [4]:
def saccade_detection(x_values, y_values, time, minlen=0.020, maxvel=20, maxacc=200):
    """Detects saccades, defined as consecutive samples with an inter-sample
    velocity of over a velocity threshold or an acceleration threshold

    arguments

    x		-	numpy array of x positions
    y		-	numpy array of y positions
    time		-	numpy array of tracker timestamps in milliseconds

    keyword arguments

    missing	-	value to be used for missing data (default = 0.0)
    minlen	-	minimal length of saccades in milliseconds; all detected
                saccades with len(sac) < minlen will be ignored
                (default = 5)
    maxvel	-	velocity threshold in pixels/second (default = 40)
    maxacc	-	acceleration threshold in pixels / second**2
                (default = 340)

    returns
    Ssac, Esac
            Ssac	-	list of lists, each containing [starttime]
            Esac	-	list of lists, each containing [starttime, endtime, duration, startx, starty, endx, endy]
    """

    # CONTAINERS
    Ssac = []
    Esac = []

    # INTER-SAMPLE MEASURES
    # the distance between samples is the square root of the sum
    # of the squared horizontal and vertical interdistances
    intdist = (np.diff(x_values) ** 2 + np.diff(y_values) ** 2) ** 0.5
    # get inter-sample times
    inttime = np.diff(time)
    # recalculate inter-sample times to seconds
    inttime =inttime / 1000

    # VELOCITY AND ACCELERATION
    # the velocity between samples is the inter-sample distance
    # divided by the inter-sample time
    vel = intdist / inttime
    # the acceleration is the sample-to-sample difference in
    # eye movement velocity
    acc = np.diff(vel)

    # SACCADE START AND END
    t0i = 0
    stop = False
    while not stop:
        # saccade start (t1) is when the velocity or acceleration
        # surpass threshold, saccade end (t2) is when both return
        # under threshold

        # detect saccade starts
        sacstarts = np.where((vel[1 + t0i:] > maxvel).astype(int) + (acc[t0i:] > maxacc).astype(int) >= 1)[0]
        if len(sacstarts) > 0:
            # timestamp for starting position
            t1i = t0i + sacstarts[0] + 1
            if t1i >= len(time) - 1:
                t1i = len(time) - 2
            t1 = time[t1i]

            # add to saccade starts
            Ssac.append([t1])

            # detect saccade endings
            sacends = np.where((vel[1 + t1i:] < maxvel).astype(int) + (acc[t1i:] < maxacc).astype(int) == 2)[0]
            if len(sacends) > 0:
                # timestamp for ending position
                t2i = sacends[0] + 1 + t1i + 2
                if t2i >= len(time):
                    t2i = len(time) - 1
                t2 = time[t2i]
                dur = t2 - t1

                # ignore saccades that did not last long enough
                if dur >= minlen:
                    # add to saccade ends
                    Esac.append([t1, t2, dur, x_values[t1i], y_values[t1i], x_values[t2i], y_values[t2i]])
                else:
                    # remove last saccade start on too low duration
                    Ssac.pop(-1)

                # update t0i
                t0i = 0 + t2i
            else:
                stop = True
        else:
            stop = True

    return Ssac, Esac



In [5]:
COLS = {"butter": ['#fce94f','#edd400','#c4a000'],
        "orange": ['#fcaf3e','#f57900','#ce5c00'],
        "chocolate": ['#e9b96e','#c17d11','#8f5902'],
        "chameleon": ['#8ae234','#73d216','#4e9a06'],
        "skyblue": ['#729fcf','#3465a4','#204a87'],
        "plum": ['#ad7fa8','#75507b','#5c3566'],
        "scarletred":['#ef2929','#cc0000','#a40000'],
        "aluminium": ['#eeeeec','#d3d7cf','#babdb6','#888a85','#555753','#2e3436']}
# FONT
FONT = {'family': 'Ubuntu', 'size': 12}
matplotlib.rc('font', **FONT)

In [6]:
def draw_display(dispsize, imagefile=True):
    """Returns a matplotlib.pyplot Figure and its axes, with a size of
    dispsize, a black background colour, and optionally with an image drawn
    onto it
    arguments
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)
    keyword arguments
    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    returns
    fig, ax		-	matplotlib.pyplot Figure and its axes: field of zeros
                    with a size of dispsize, and an image drawn onto it
                    if an imagefile was passed
    """

    # construct screen (black background)
    screen = np.zeros((dispsize[1], dispsize[0], 4), dtype='float32')
    # if an image location has been passed, draw the image
    if imagefile != None:
        # check if the path to the image exists
        #if not os.path.isfile(imagefile):
            #raise Exception("ERROR in draw_display: imagefile not found at '%s'" % imagefile)
        # load image
        #img = image.imread(imagefile)
        img = plt.imread(imagefile)
        #img = PIL.Image.open(imagefile)

        # width and height of the image
        w, h = len(img[0]), len(img)
        #w,h=img.size
        # x and y position of the image on the display
        x = dispsize[0] // 2 - w // 2
        y = dispsize[1] // 2 - h // 2
        # draw the image on the screen
        screen[y:y + h, x:x + w, :] += img
    # dots per inch
    dpi = 100.0
    # determine the figure size in inches
    figsize = (dispsize[0] / dpi, dispsize[1] / dpi)
    # create a figure
    fig = plt.figure(figsize=figsize, dpi=dpi, frameon=False)
    ax = plt.Axes(fig, [0, 0, 1, 1])
    ax.set_axis_off()
    fig.add_axes(ax)
    # plot display
    ax.axis([0, dispsize[0], 0, dispsize[1]])
    ax.imshow(screen)  # , origin='upper')

    return fig ,ax

In [7]:
def parse_fixations(fixations):
    """Returns all relevant data from a list of fixation ending events

    arguments

    fixations		-	a list of fixation ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Efix']

    returns

    fix		-	a dict with three keys: 'x', 'y', and 'dur' (each contain
                a numpy array) for the x and y coordinates and duration of
                each fixation
    """

    # empty arrays to contain fixation coordinates
    fix = {'x': np.zeros(len(fixations)),
           'y': np.zeros(len(fixations)),
           'dur': np.zeros(len(fixations))}
    # get all fixation coordinates
    for fixnr in range(len(fixations)):
        stime, etime, dur, ex, ey = fixations[fixnr]
        fix['x'][fixnr] = ex
        fix['y'][fixnr] = ey
        fix['dur'][fixnr] = dur

    return fix


In [8]:
def draw_fixations(fixations, dispsize, imagefile=True, durationsize=False, durationcolour=False, alpha=0.5,
                   savefilename=True):
    
    """Draws circles on the fixation locations, optionally on top of an image,
    with optional weigthing of the duration for circle size and colour

    arguments

    fixations		-	a list of fixation ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Efix']
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)

    keyword arguments

    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    durationsize	-	Boolean indicating whether the fixation duration is
                    to be taken into account as a weight for the circle
                    size; longer duration = bigger (default = True)
    durationcolour	-	Boolean indicating whether the fixation duration is
                    to be taken into account as a weight for the circle
                    colour; longer duration = hotter (default = True)
    alpha		-	float between 0 and 1, indicating the transparancy of
                    the heatmap, where 0 is completely transparant and 1
                    is completely untransparant (default = 0.5)
    savefilename	-	full path to the file in which the heatmap should be
                    saved, or None to not save the file (default = None)

    returns

    fig			-	a matplotlib.pyplot Figure instance, containing the
                    fixations
    """
    fig, ax = draw_display(dispsize, imagefile=imagefile)
    for i in range(len(fixations)):
        
        
       
    # FIXATIONS
        fix = parse_fixations([fixations[i]])

    # IMAGE
        

    # CIRCLES
    # duration weigths
        if durationsize:
            #siz = 1 * (fix['dur'] / 30.0)
            siz = round(round(1 * (fix['dur'],5)) * 10000)
        else:
            #siz = 1 * np.median(fix['dur'] / 30.0)
            siz = round(round(1 * np.median(fix['dur']),5) * 10000)
            # if siz> 450.:
                    #ax.scatter(fix['x'], fix['y'], s=siz, c=col, marker='o', cmap='jet', alpha=alpha, edgecolors='none')

        if durationcolour:
            col = fix['dur']
        else:
            col = COLS['chameleon'][2]
        # draw circles
        ax.scatter(fix['x'], fix['y'], s=siz, c=col, marker='o', cmap='jet', alpha=alpha, edgecolors='none')
        if siz > 450:
            ax.annotate(str(siz),(fix['x'],fix['y']),color='black', ha='center')

        # FINISH PLOT
        # invert the y axis, as (0,0) is top left on a display
    ax.invert_yaxis()
        # save the figure if a file name was provided
    if savefilename != None:
        fig.savefig(savefilename)

    return fig


In [9]:
def draw_scanpath(fixations, saccades, dispsize, imagefile=True, alpha=0.1, savefilename=True):
    
    """Draws a scanpath: a series of arrows between numbered fixations,
    optionally drawn over an image

    arguments

    fixations		-	a list of fixation ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Efix']
    saccades		-	a list of saccade ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Esac']
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)

    keyword arguments

    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    alpha		-	float between 0 and 1, indicating the transparancy of
                    the heatmap, where 0 is completely transparant and 1
                    is completely untransparant (default = 0.5)
    savefilename	-	full path to the file in which the heatmap should be
                    saved, or None to not save the file (default = None)

    returns

    fig			-	a matplotlib.pyplot Figure instance, containing the
                    heatmap
    """

    # image
    fig,ax  = draw_display(dispsize, imagefile=imagefile)
    Fixt = []
    # FIXATIONS
    # parse fixations
    for i in range(len(fixations)):
        
        fix = parse_fixations([fixations[i]])
        Fixt.append(fix)
        # draw fixations
        ax.scatter(fix['x'], fix['y'], s=round(round(1 * np.median(fix['dur']),5) * 10000), c=COLS['chameleon'][2], marker='o', cmap='jet',
                   alpha=alpha, edgecolors='none')
        # draw annotations (fixation numbers)
    for i in range(len(Fixt)):

        ax.annotate(str(i + 1), (Fixt[i]['x'], Fixt[i]['y']), color='Red', alpha=1,
                    horizontalalignment='center', verticalalignment='center') #multialignment='center')
    #for j in range(len(Fixt)):
        x_va = []
        y_va = []
        for i in range(len(Fixt)):
            x_va.append(Fixt[i]['x'][0])
            y_va.append(Fixt[i]['y'][0])
            
        different_x = []
        different_y = []
        for i in range(1,len(Fixt)):
            a = x_va[i] - x_va[i-1]
            b = y_va[i] - y_va[i-1]
            different_x.append(a)
            different_y.append(b)
        different_x.append(0.0)
        different_y.append(0.0)
        for k in range(len(x_va)):
            
            ax.arrow(x_va[k],y_va[k],different_x[k],different_y[k],fc=COLS['aluminium'][0], ec= None, fill=True,
                     shape='full', width=5, head_width=8,alpha=alpha, head_starts_at_zero=False, overhang=0)


    # SACCADES
        #if saccades:
            #for j in range(len(saccades)):
                
            # loop through all saccades
                #for st, et, dur, sx, sy, ex, ey in [saccades[j]]:
                    # draw an arrow between every saccade start and ending
                    #ax.arrow(sx, sy, ex - sx, ey - sy, alpha=alpha, fc=COLS['aluminium'][0], ec=COLS['aluminium'][5], fill=True,
                             #shape='full', width=10, head_width=20, head_starts_at_zero=False, overhang=0)

    # invert the y axis, as (0,0) is top left on a display
    ax.invert_yaxis()
    # save the figure if a file name was provided
    if savefilename != None:
        fig.savefig(savefilename)

    return fig,Fixt

In [10]:
def gen_heatmap(click_data, width, height, radius):
    heatmap = np.zeros((height, width), dtype=np.int32)
    mask = np.ones((height, width), dtype=np.int32)
    
    for x, y in tqdm(click_data):
        Y, X = np.ogrid[:height, :width]
        dist_from_center = np.sqrt((X - x)**2 + (Y-y)**2)
        current_mask = dist_from_center < radius
        heatmap[current_mask] = heatmap[current_mask] + mask[current_mask]
        
    return heatmap

In [11]:



dimension = (1080,1920)

number_of_clicks = 1000

time_filter=df_eyetracking["time"][0]

#iterating over images and thier start and end time
#file_no = 1
for idx, row in tqdm(df_cutting.iterrows(), total=len(df_cutting)):
    image = row["ImagePath"]
    start_time = row["StartTime"]
    end_time = row["EndTime"]
    
    df_tmp = df_eyetracking[(df_eyetracking["time"] >= start_time) & (df_eyetracking["time"] <= end_time)]
    #print (df_tmp.head())
    #accepting only valid values form eye tracking data
    #print (df_tmp.head())
    
    x_values_l = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["l_display_x"].values
    y_values_l = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)] ["l_display_y"].values

    x_values_r = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_display_x"].values
    y_values_r = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_display_y"].values
    
    #time=df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]['time'].values
    #print (time)
    
    df_timetemp = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]
    time = df_timetemp['time'].values.tolist()
    #print (time)
    

    #taking average of x_value_l,x_value_r
    
    x_values = [ int((x_l+x_r)/2.0) for (x_l, x_r) in zip(x_values_l, x_values_r)]
    y_values = [ int((y_l+y_r)/2.0) for (y_l, y_r) in zip(y_values_l, y_values_r)]
    imageName = image
    algorithm_name = imageName.split("/")[-1]
    
    
    Sfix, Efix= fixation_detection(x_values, y_values, time, maxdist=25, mindur=0.020)
    #print (Efix)
    
    fig= draw_fixations(Efix, (1920,1080), imagefile=image, durationsize=False, durationcolour=False, alpha=0.5,
                   savefilename="./{}/fixations_{}".format(folders[1],algorithm_name))
    
    Ssac, Esac=saccade_detection(x_values, y_values, time, minlen=0.020, maxvel=20, maxacc=300)
    
    #print (Esac)
    fig,Fixt = draw_scanpath(Efix, Esac, (1920,1080), imagefile=image, alpha=0.5, 
                             savefilename="./{}/scanpath_{}".format(folders[2],algorithm_name))
    
    
    
    
    click_data = list(zip([int(x) for x in x_values], [int(y) for y in y_values]))
    
    #image = image.replace("./snippets/Final","./snippets")
    
    image = PIL.Image.open(imageName)
    w, h = image.size
    heatmap= gen_heatmap(click_data, w, h, 50)
    
    
    #perform gaussian blur
    cmap = plt.cm.get_cmap("jet") 
    heatmap = gaussian_filter(heatmap, sigma=5)
    heatmap = cmap(heatmap)
    plt.imshow(heatmap, cmap='jet', interpolation='nearest')
    image.resize((h, w))
    image = np.array(image)
    heatmap=heatmap*255
    heatmap=heatmap.astype(int)
    heatmap_image =0.7*image + 0.3*heatmap
    heatmap_image = heatmap_image
    
    print (algorithm_name)
    #image_name = f"./{participant_name}/heatmap_{algorithm_name}"
    image_name = "./{}/heatmap_{}.png".format(folders[0],algorithm_name)
    im = PIL.Image.fromarray(np.uint8(heatmap_image))
    im.save(image_name)
    plt.close('all')
    #file_no = file_no + 1
    

findfont: Font family ['Ubuntu'] not found. Falling back to DejaVu Sans.



ArraySum(CNL).png



ArraySumresp(CNL).png



BubbleSort(WIC).png



BubbleSortresp(WIC).png


...


BinaryConversionresp(WOC).png



MultiplyMatrix(CNL).png



MultiplyMatrixresp(CNL).png



PrimeNumber(WIC).png



PrimeNumberresp(WIC).png

